In [1]:
import os
import pandas as pd
from interpret import show
from interpret.provider import InlineProvider
from interpret.glassbox import ExplainableBoostingRegressor
from sklearn.model_selection import train_test_split
from interpret.perf import RegressionPerf

In [2]:
in_file = r"E:\work\results\grid_search\metrics.csv"
in_file = os.path.abspath(in_file)
in_dir = os.path.dirname(in_file)
random_state = 42

df = pd.read_csv(in_file)

df

,image,class,threshold,metric,value,data_dataset,model_activation,model_depth,model_in_channels,model_out_classes,model_residual_units,model_width,model_width_ratio,job_hash,epoch,loss,predict_time
0,bremen_000075_000019,road,0.5,iou,0.927125,cityscapes,piecewise_linear,8,3,19,2,50,1.4,025cb7327dfa11ba27fd6fafcec81e13,237,0.326670,611.571244
1,bremen_000075_000019,sidewalk,0.5,iou,0.860142,cityscapes,piecewise_linear,8,3,19,2,50,1.4,025cb7327dfa11ba27fd6fafcec81e13,237,0.326670,611.571244
2,bremen_000075_000019,building,0.5,iou,0.842418,cityscapes,piecewise_linear,8,3,19,2,50,1.4,025cb7327dfa11ba27fd6fafcec81e13,237,0.326670,611.571244
3,bremen_000075_000019,fence,0.5,iou,0.534923,cityscapes,piecewise_linear,8,3,19,2,50,1.4,025cb7327dfa11ba27fd6fafcec81e13,237,0.326670,611.571244
4,bremen_000075_000019,pole,0.5,iou,0.525356,cityscapes,piecewise_linear,8,3,19,2,50,1.4,025cb7327dfa11ba27fd6fafcec81e13,237,0.326670,611.571244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628283,lindau_000018_000019,vegetation,0.5,recall,0.833764,cityscapes,cos,4,3,19,4,30,1.2,fe92838c1edac48fb3f28d5691c0e6d0,48,0.381132,624.750310
2628284,lindau_000018_000019,building,0.5,precision,0.864564,cityscapes,cos,4,3,19,4,30,1.2,fe92838c1edac48fb3f28d5691c0e6d0,48,0.381132,624.750310
2628285,lindau_000018_000019,pole,0.5,precision,0.023660,cityscapes,cos,4,3,19,4,30,1.2,fe92838c1edac48fb3f28d5691c0e6d0,48,0.381132,624.750310
2628286,lindau_000018_000019,traffic sign,0.5,precision,0.730199,cityscapes,cos,4,3,19,4,30,1.2,fe92838c1edac48fb3f28d5691c0e6d0,48,0.381132,624.750310


In [3]:
predict_df = df[["job_hash", 'model_depth', 'model_residual_units', 'model_width', 'model_width_ratio', 'model_activation', 'loss']].drop_duplicates()
predict_df["loss"] = -predict_df["loss"]
predict_df

,job_hash,model_depth,model_residual_units,model_width,model_width_ratio,model_activation,loss
0,025cb7327dfa11ba27fd6fafcec81e13,8,2,50,1.4,piecewise_linear,-0.326670
18252,025e9ee4f5c6ff1fbb9a04e53c6fa7e1,6,4,30,1.4,swish,-0.307142
36504,0745dd0ff6028bec00aedab0ee132b35,6,2,30,1.4,cos,-0.368619
54756,076dec8f40a2bde8ee73ad5dd9fcc099,4,4,50,1.4,swish,-0.328748
73008,083ba7cbeb18ac74441387a9255f6e80,6,2,50,1.2,cos,-0.368678
...,...,...,...,...,...,...,...
2537028,f8c9dfa56064aef37791c5784daf4e9b,6,4,30,1.4,piecewise_linear,-0.327602
2555280,fbf7dda46ab6eb773a67d076dafe5555,6,2,50,1.4,leaky_relu,-0.335650
2573532,fcada0d2b1fd9e5fdbc343f967df52cb,4,2,30,1.2,swish,-0.435853
2591784,fe3916a23df4e9ed456b4a9b2ac15fa6,4,2,30,1.2,leaky_relu,-0.415647


In [4]:
train_cols = ['model_depth', 'model_residual_units', 'model_width', 'model_width_ratio', 'model_activation']
label = "loss"

X = predict_df[train_cols]
y = predict_df[label]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=random_state)

In [5]:
ebm = ExplainableBoostingRegressor(feature_names=train_cols,
                              feature_types=['categorical'] * len(train_cols),
                              random_state=random_state, interactions=0)
ebm.fit(X_train, y_train)

ExplainableBoostingRegressor(feature_names=['model_depth',
                                            'model_residual_units',
                                            'model_width', 'model_width_ratio',
                                            'model_activation'],
                             feature_types=['categorical', 'categorical',
                                            'categorical', 'categorical',
                                            'categorical'],
                             interactions=0)

In [6]:
ebm_global = ebm.explain_global(name="Training Epochs Estimator")
ebm_local = ebm.explain_local(X_test, y_test, name="Training Epochs Estimator")
ebm_perf = RegressionPerf(ebm.predict).explain_perf(X_test, y_test, name='Training Epochs Estimator')
show([ebm_global, ebm_local, ebm_perf])

<!-- http://127.0.0.1:7722/2341451786888/ -->
 Open in new window

In [7]:
ebm_score = ebm.score(X_test, y_test)
ebm_score

0.8079932793288155